In [25]:
import undetected_chromedriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import ddddocr

from modules.chrome import Chrome

In [26]:
# 初始化
URL = "https://irs.thsrc.com.tw/IMINT/"

options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
chrome = Chrome(options=options)
chrome.set_window_size(1600, 1000)

chrome.get(URL)

In [27]:
# 同意 cookie
try:
    chrome.find_element_and_wait(
        By.XPATH, "//*[@id='cookieAccpetBtn']", 1000).click()
except:
    pass

In [28]:
# 輸入訂票資訊
FROM = u"桃園"
TO = u"新竹"
ADULT = u"0"
CHILD = u"0"
HEART = u"0"
ELDER = u"0"
STUDENT = u"1"
DATE = u"2023/05/31"
TIME = u"10:00"

Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[3]/div[1]/div/div[1]/div/select")).select_by_visible_text(FROM)
Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[3]/div[1]/div/div[2]/div/select")).select_by_visible_text(TO)
Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[4]/div[1]/div[1]/div/select")).select_by_visible_text(ADULT)
Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[4]/div[1]/div[2]/div/select")).select_by_visible_text(CHILD)
Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[4]/div[1]/div[3]/div/select")).select_by_visible_text(HEART)
Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[4]/div[1]/div[4]/div/select")).select_by_visible_text(ELDER)
Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[4]/div[1]/div[5]/div/select")).select_by_visible_text(STUDENT)
chrome.execute_script(f"document.getElementById('toTimeInputField').setAttribute('value', '{DATE}')")
# chrome.find_element(By.XPATH,
#                     "//*[@id='toTimeInputField']").send_keys(DATE)
Select(chrome.find_element(By.XPATH,
       "//*[@id='BookingS1Form']/div[3]/div[2]/div/div[2]/div[1]/select")).select_by_visible_text(TIME)

In [30]:
# 驗證碼
chrome.find_element(
    By.ID, "BookingS1Form_homeCaptcha_passCode").screenshot("captcha.png")
with open("captcha.png", "rb") as f:
    image_bytes = f.read()
security_code = ddddocr.DdddOcr().classification(image_bytes)
security_code_blank = chrome.find_element(By.ID, "securityCode")
security_code_blank.clear()
security_code_blank.send_keys(security_code)

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i


In [31]:
# 提交表單
chrome.find_element(By.ID, "SubmitButton").click()

In [32]:
# 取得班次
class Car:
    def __init__(self, departure: str, arrival: str, code: str, discount: str) -> None:
        self.departure = departure
        self.arrival = arrival
        self.code = int(code)
        self.discount = "None" if (discount == "") else discount

car_list = list(map(lambda e: Car(e[0], e[1], e[2], e[3]), zip(
    map(lambda e: e.text, chrome.find_elements(By.ID, "QueryDeparture")),
    map(lambda e: e.text, chrome.find_elements(By.ID, "QueryArrival")),
    map(lambda e: e.text, chrome.find_elements(By.ID, "QueryCode")),
    map(lambda e: e.text.replace("\n", ","), chrome.find_elements(By.CSS_SELECTOR, "div[class='discount uk-flex']")))))

for i in range(len(car_list)):
    car = car_list[i]
    print(f"{i + 1:2d} {car.code:4d} {car.departure} {car.arrival} {car.discount}")

 1  621 10:10 10:21 早鳥9折,學生75折
 2  817 10:34 10:45 早鳥8折,學生75折
 3  625 10:43 10:53 早鳥8折,學生75折
 4  627 11:10 11:21 早鳥9折,學生75折
 5  821 11:34 11:45 早鳥9折,學生75折
 6  633 12:10 12:21 早鳥9折,學生88折
 7  825 12:34 12:45 早鳥9折,學生75折
 8  639 13:10 13:21 學生88折
 9  829 13:34 13:45 早鳥9折,學生75折
10  833 14:34 14:45 早鳥9折,學生75折


In [33]:
# 選擇班次
SELECTED = 3
chrome.find_elements(By.CSS_SELECTOR, "input[type='radio']")[SELECTED].click()

In [34]:
# 提交確認車次
chrome.find_element(By.XPATH, "//*[@id='BookingS2Form']/section[2]/div/div/input").click()

In [35]:
# 輸入身份資訊
ID = "J123456789"
PHONE = "0911222333"
EMAIL = "abc@gmail.com"

id_blank = chrome.find_element(By.ID, "idNumber")
id_blank.clear()
id_blank.send_keys(ID)
phone_blank = chrome.find_element(By.ID, "mobilePhone")
phone_blank.clear()
phone_blank.send_keys(PHONE)
email_blank = chrome.find_element(By.ID, "email")
email_blank.clear()
email_blank.send_keys(EMAIL)
chrome.find_element(By.XPATH, "//*[@id='BookingS3FormSP']/section[2]/div[3]/div[1]/label/input").click()

In [36]:
# 完成訂票
chrome.find_element(By.ID, "isSubmit").click()

In [37]:
# 關閉學生優惠提醒視窗
try:
    chrome.find_element_and_wait(By.ID, "step4StudentModalBtn", 800).click()
except:
    pass

In [38]:
# 截圖
chrome.find_element(By.XPATH, "//*[@id='BookingS4Form']/section[1]").screenshot("result.png")

True